In [1]:
from lightgbm.sklearn import LGBMRanker
import lightgbm as lgb
from datetime import timedelta
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from datetime import datetime, date, timedelta

In [38]:
INPUT_DIR = '../input/'
OUTPUT_DIR = '../output/'
INTERMED_DIR = '../intermed/'

In [48]:
submit_df = pd.read_csv(os.path.join(INTERMED_DIR, 'submission.csv'))
submit_df.shape

(1371980, 2)

In [49]:
result = pd.read_csv(os.path.join(INTERMED_DIR, 'result70.csv'))
result.head()

,customer_id,prediction
0,00d7ebd46f6a6d53630d41386b6ef6a505cdc4c80011ff...,0918522001 0915526001 0881942002 0914805002 08...
1,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0901955001 0861477001 0913030001 0776237031 08...
2,013f00f9e218549246a3aa82b3f3a0c22a693bc25fa735...,0839402002 0857621002 0857621001 0843284001 08...
3,01bada2a453b09c70ea57bdda5a9ef0fb04062718d3a3d...,0724906006 0914441001 0723469001 0820572001 09...
4,01dd96059a11759518f10969d0a528f03c8501dc4c628b...,0859118001 0910872001 0876053003 0817353003 08...


In [50]:
submit_df = pd.merge(submit_df, result, on='customer_id', how='outer')
submit_df.head()

,customer_id,prediction_x,prediction_y
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0751471001 0924243001 04...,NaN
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0924243001 0924243002 0739590027 08...,NaN
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0924243001 0924243002 07...,NaN
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0924243002 0924243001 0751471001 05...,NaN
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0927530004 0896152002 09...,NaN


In [51]:
submit_df.prediction_y.isnull().sum()

1356522

In [52]:
result.shape

(15458, 2)

In [53]:
len(submit_df)

1371980

In [54]:
submit_df = submit_df.fillna(-1)
submit_df['prediction'] = submit_df.apply(
    lambda x: x['prediction_y'] if x['prediction_y'] != -1 else x['prediction_x'], axis=1)
submit_df.head()

,customer_id,prediction_x,prediction_y,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0751471001 0924243001 04...,-1,0568601043 0568601006 0751471001 0924243001 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0924243001 0924243002 0739590027 08...,-1,0826211002 0924243001 0924243002 0739590027 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0924243001 0924243002 07...,-1,0794321007 0852643001 0924243001 0924243002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0924243002 0924243001 0751471001 05...,-1,0448509014 0924243002 0924243001 0751471001 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0927530004 0896152002 09...,-1,0730683050 0791587015 0927530004 0896152002 09...


In [55]:
submit_df = submit_df.drop(columns=['prediction_y', 'prediction_x'])
submit_df.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0751471001 0924243001 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0924243001 0924243002 0739590027 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0924243001 0924243002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0924243002 0924243001 0751471001 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0927530004 0896152002 09...


In [57]:
import datetime
now = datetime.datetime.now()
now = now.strftime("%m%d_%H%M")

submit_df.to_csv(os.path.join(OUTPUT_DIR, f'predictions/submisssion_ranking_{now}.csv'), index = False)